# Importing libraries

In [1]:
import pandas as pd                                 # reading the .CSV file I/O (e.g. pd.read_csv)
import numpy as np                                  # data manipulation
from bs4 import BeautifulSoup                       # for pulling data out of HTML and XML files.
from sklearn.feature_extraction.text import CountVectorizer 
                                                    #extract features in a format supported by ML algorithms(such as text)
from sklearn.ensemble import RandomForestClassifier # for classification and prediction 
import nltk                                         # removes unnecessary words from dataset
from nltk.corpus import stopwords
import os
import re 

c:\python\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\python\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\python\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Read the dataset

In [2]:
train = pd.read_csv("labeledTrainData.tsv", header = 0, delimiter = '\t')
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeledTrain = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

In [3]:
print(train.shape)
train.columns.values

(25000, 3)


array(['id', 'sentiment', 'review'], dtype=object)

In [4]:
#print the first review is:
print(train["review"][0])

With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally star

In [5]:

train1 = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
print(train1.shape)
train1.columns.values

(50000, 2)


array(['id', 'review'], dtype=object)

In [6]:
print(train1["id"][0])
print(train1["review"][0])

"9999_0"
"Watching Time Chasers, it obvious that it was made by a bunch of friends. Maybe they were sitting around one day in film school and said, \"Hey, let's pool our money together and make a really bad movie!\" Or something like that. What ever they said, they still ended up making a really bad movie--dull story, bad script, lame acting, poor cinematography, bottom of the barrel stock music, etc. All corners were cut, except the one that would have prevented this film's release. Life's like that."


# Clean the training data

In [7]:
# from nltk.corpus import stopwords
# making the function of Cleaning and parsing the training set movie reviews...
def review_to_words( raw_review):
        review_text = BeautifulSoup(raw_review, features="html5lib").get_text()
        letters_only = re.sub("[^a-zA-Z]", " ", review_text)            
        words = letters_only.lower().split() 
        stops = set(stopwords.words("english"))                         
        meaningful_words = [w for w in words if not w in stops]         
        return(" ".join( meaningful_words ))

In [8]:
# Cleaning and parsing the training data set movie reviews...
num_reviews = train["review"].size
clean_train_reviews = []

for i in range( 0, num_reviews):
        clean_train_reviews.append( review_to_words( train["review"][i]))

# Create the bag of words

In [9]:
# from sklearn.feature_extraction.text import CountVectorizer 
# tokenizer counts the freq.of the each worrd and create the dictionary of them.
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

# Train the classifier

In [10]:
# Training the random forest 
forest = RandomForestClassifier(n_estimators = 150) 
forest = forest.fit( train_data_features, train["sentiment"] )

# Format the testing data

In [11]:
# Cleaning and parsing the testing data set movie reviews...
num_reviews = len(test["review"])
clean_test_reviews = [] 

for i in range(0,num_reviews):
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )
    
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()


# Predict reviews in the testing data

In [12]:
# Predicting test labels...
# writing the results in test_data_predict_model.csv 
result = forest.predict(test_data_features)
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

output.to_csv( "test_data_predict_model.csv", index=False, quoting=3 )

# Format the unlabeledTrain data

In [13]:
print(unlabeledTrain.shape)
unlabeledTrain.columns.values

(50000, 2)


array(['id', 'review'], dtype=object)

In [14]:
print(unlabeledTrain["id"][0])
print(unlabeledTrain["review"][0])

"9999_0"
"Watching Time Chasers, it obvious that it was made by a bunch of friends. Maybe they were sitting around one day in film school and said, \"Hey, let's pool our money together and make a really bad movie!\" Or something like that. What ever they said, they still ended up making a really bad movie--dull story, bad script, lame acting, poor cinematography, bottom of the barrel stock music, etc. All corners were cut, except the one that would have prevented this film's release. Life's like that."


In [16]:
# Cleaning and parsing the unlabeledTrain data set movie reviews...
unlabeledTrain_num_reviews = len(unlabeledTrain["review"])
clean_unlabeledTrain_reviews = [] 

for i in range(0,unlabeledTrain_num_reviews):
    clean_review = review_to_words( unlabeledTrain["review"][i] )
    clean_unlabeledTrain_reviews.append( clean_review )
    
unlabeledTrain_data_features = vectorizer.transform(clean_unlabeledTrain_reviews)
unlabeledTrain_data_features = unlabeledTrain_data_features.toarray()


# Predict reviews in the unlabeledTrain data

In [17]:
# Predicting unlabeledTrain labels...
# writing the results in unlabeledTrain_output.csv
result1 = forest.predict(unlabeledTrain_data_features)
output = pd.DataFrame( data={"id":unlabeledTrain["id"], "sentiment":result1} )

output.to_csv( "unlabeledTrain_output.csv", index=False, quoting=3 )